In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('../../../data/WA_Fn-UseC_-Telco-Customer-Churn.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [3]:
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')

In [4]:
# df['TotalCharges'].str.astype(float)

In [5]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [6]:
#label encode Churn
#binary encode the yes/no
#one hot encode the rest

In [7]:
# sns.pairplot(df)

In [8]:
df.columns = df.columns.str.lower()

In [9]:
for col in df.columns:
    print(col,df[col].nunique())

customerid 7043
gender 2
seniorcitizen 2
partner 2
dependents 2
tenure 73
phoneservice 2
multiplelines 3
internetservice 3
onlinesecurity 3
onlinebackup 3
deviceprotection 3
techsupport 3
streamingtv 3
streamingmovies 3
contract 3
paperlessbilling 2
paymentmethod 4
monthlycharges 1585
totalcharges 6530
churn 2


In [10]:
cust_id = df['customerid']
df = df.drop('customerid',axis=1)

df['totchg_per_tenure'] = df['totalcharges'] / df['tenure']
df['monthly_div_tot'] = df['monthlycharges'] / df['totalcharges']


In [11]:
ord_encode = ['gender','partner','dependents','phoneservice','multiplelines','internetservice','onlinesecurity','onlinebackup',
              'deviceprotection','techsupport','streamingtv','streamingmovies','contract','paperlessbilling','paymentmethod']
labelencode =['churn']
num = ['seniorcitizen','tenure','monthlycharges','totalcharges','totchg_per_tenure','monthly_div_tot']

In [12]:
X = df[ord_encode].join(df[num],how='left')
X = X.join(cust_id,how='left')
y = df[labelencode]

from sklearn.model_selection import train_test_split

X_t, X_test, y_t, y_test = train_test_split(X,y,random_state=0)

In [13]:
X_train, X_val, y_train, y_val = train_test_split(X_t,y_t,random_state=0)

Impute missing values as 0

In [14]:
from sklearn.impute import SimpleImputer

impute = SimpleImputer(strategy='constant',fill_value=0)
X_train_im = impute.fit_transform(X_train)
X_val_im = impute.transform(X_val)

X_train_im = pd.DataFrame(X_train_im,columns=X.columns)
X_val_im = pd.DataFrame(X_val_im,columns=X.columns)

In [24]:
X_train_im.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3961 entries, 0 to 3960
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   gender             3961 non-null   object
 1   partner            3961 non-null   object
 2   dependents         3961 non-null   object
 3   phoneservice       3961 non-null   object
 4   multiplelines      3961 non-null   object
 5   internetservice    3961 non-null   object
 6   onlinesecurity     3961 non-null   object
 7   onlinebackup       3961 non-null   object
 8   deviceprotection   3961 non-null   object
 9   techsupport        3961 non-null   object
 10  streamingtv        3961 non-null   object
 11  streamingmovies    3961 non-null   object
 12  contract           3961 non-null   object
 13  paperlessbilling   3961 non-null   object
 14  paymentmethod      3961 non-null   object
 15  seniorcitizen      3961 non-null   object
 16  tenure             3961 non-null   object


In [31]:
from sklearn.preprocessing import OrdinalEncoder
#ord encode categoticals
ordi = OrdinalEncoder(dtype=np.int)

X_train_ord = ordi.fit_transform(X_train_im[ord_encode])
X_val_ord = ordi.transform(X_val_im[ord_encode])

X_train_ord = pd.DataFrame(X_train_ord,columns=X[ord_encode].columns)
X_val_ord = pd.DataFrame(X_val_ord,columns=X[ord_encode].columns)

from sklearn.preprocessing import StandardScaler
#scale numerical
#use median for limit effect of outliers
ss = StandardScaler()

X_train_num = ss.fit_transform(X_train_im[num])
X_val_num = ss.transform(X_val_im[num])

X_train_num = pd.DataFrame(X_train_num,columns=X[num].columns)
X_val_num = pd.DataFrame(X_val_num,columns=X[num].columns)

#add customer id
X_train_num = X_train_num.join(X_train_im['customerid'],how='right')
X_val_num = X_val_num.join(X_val_im['customerid'],how='right')

In [32]:
X_val_num

,seniorcitizen,tenure,monthlycharges,totalcharges,totchg_per_tenure,monthly_div_tot,customerid
0,-0.427069,0.022881,-1.489476,-0.712634,-1.471300,-0.449101,3769-MHZNV
1,-0.427069,-0.344542,0.547812,-0.157005,0.518759,-0.405672,4277-UDIEF
2,-0.427069,-1.120212,-0.009181,-0.853459,0.198523,0.097378,7905-TVXTA
3,-0.427069,-1.283510,0.313641,-0.976974,0.313909,3.027120,8687-BAFGU
4,-0.427069,1.370097,-0.071069,0.823734,-0.062752,-0.502407,8404-VIOMB
...,...,...,...,...,...,...,...
1316,-0.427069,-1.161036,-1.504530,-0.979833,-1.585227,0.485920,0690-SRQID
1317,-0.427069,0.349479,-1.516238,-0.667400,-1.520334,-0.467010,5465-BUBFA
1318,-0.427069,-0.875263,0.721767,-0.613853,0.553328,-0.210706,3797-FKOGQ
1319,-0.427069,1.615045,0.766929,1.855095,0.838168,-0.508011,4077-HWUYD


In [33]:
#combine the categorical and numerical dfs

X_train_sc = X_train_ord.join(X_train_num,how='left')
X_val_sc = X_val_ord.join(X_val_num,how='left')

In [34]:
#encode the labels

from sklearn.preprocessing import LabelEncoder

lbl = LabelEncoder()

y_train = lbl.fit_transform(np.array(y_train))
y_val = lbl.transform(y_val)

y_train_enc = pd.DataFrame(y_train,columns=y.columns)
y_val_enc = pd.DataFrame(y_val,columns=y.columns)

In [35]:
y.value_counts()

churn
No       5174
Yes      1869
dtype: int64

In [36]:
X_train_sc.to_csv('X_train_sc.csv',index=False)
X_val_sc.to_csv('X_val_sc.csv',index=False)

y_train_enc.to_csv('y_train_enc.csv',index=False)
y_val_enc.to_csv('y_val_enc.csv',index=False)

In [38]:
X_val_sc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1321 entries, 0 to 1320
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   gender             1321 non-null   int32  
 1   partner            1321 non-null   int32  
 2   dependents         1321 non-null   int32  
 3   phoneservice       1321 non-null   int32  
 4   multiplelines      1321 non-null   int32  
 5   internetservice    1321 non-null   int32  
 6   onlinesecurity     1321 non-null   int32  
 7   onlinebackup       1321 non-null   int32  
 8   deviceprotection   1321 non-null   int32  
 9   techsupport        1321 non-null   int32  
 10  streamingtv        1321 non-null   int32  
 11  streamingmovies    1321 non-null   int32  
 12  contract           1321 non-null   int32  
 13  paperlessbilling   1321 non-null   int32  
 14  paymentmethod      1321 non-null   int32  
 15  seniorcitizen      1321 non-null   float64
 16  tenure             1321 